In [1]:
import pandas as pd
import numpy as np 

In [2]:
df = pd.read_excel('/kaggle/input/insurance-fraud-detection/Worksheet in Case Study question 2.xlsx')

In [3]:
# In this dataset missing values have been denoted by '?'
# we are replacing ? with NaN for them to be imputed down the line.
df =df.replace('?',np.nan)

In [4]:
df.shape

(1000, 39)

In [5]:
df.isna().sum()

months_as_customer               0
age                              0
policy_number                    0
policy_bind_date                 0
policy_state                     0
policy_csl                       0
policy_deductable                0
policy_annual_premium            0
umbrella_limit                   0
insured_zip                      0
insured_sex                      0
insured_education_level          0
insured_occupation               0
insured_hobbies                  0
insured_relationship             0
capital-gains                    0
capital-loss                     0
incident_date                    0
incident_type                    0
collision_type                 178
incident_severity                0
authorities_contacted           91
incident_state                   0
incident_city                    0
incident_location                0
incident_hour_of_the_day         0
number_of_vehicles_involved      0
property_damage                360
bodily_injuries     

In [6]:
columns_to_drop = [
    'policy_number', 'policy_bind_date', 'insured_zip', 'insured_hobbies', 
    'incident_date', 'capital-gains', 'capital-loss', 'witnesses', 
    'vehicle_claim', 'property_claim', 'injury_claim', 'auto_make', 
    'auto_model', 'auto_year', 'authorities_contacted', 'incident_location'
]


In [7]:
df = df.drop(columns=columns_to_drop)

In [8]:
df.isna().sum()

months_as_customer               0
age                              0
policy_state                     0
policy_csl                       0
policy_deductable                0
policy_annual_premium            0
umbrella_limit                   0
insured_sex                      0
insured_education_level          0
insured_occupation               0
insured_relationship             0
incident_type                    0
collision_type                 178
incident_severity                0
incident_state                   0
incident_city                    0
incident_hour_of_the_day         0
number_of_vehicles_involved      0
property_damage                360
bodily_injuries                  0
police_report_available        343
total_claim_amount               0
fraud_reported                   0
dtype: int64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   months_as_customer           1000 non-null   int64  
 1   age                          1000 non-null   int64  
 2   policy_state                 1000 non-null   object 
 3   policy_csl                   1000 non-null   object 
 4   policy_deductable            1000 non-null   int64  
 5   policy_annual_premium        1000 non-null   float64
 6   umbrella_limit               1000 non-null   int64  
 7   insured_sex                  1000 non-null   object 
 8   insured_education_level      1000 non-null   object 
 9   insured_occupation           1000 non-null   object 
 10  insured_relationship         1000 non-null   object 
 11  incident_type                1000 non-null   object 
 12  collision_type               822 non-null    object 
 13  incident_severity  

In [10]:
# Separate columns into different types
num_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Categorical columns (object type columns with more than two unique values)
categorical_columns = df.select_dtypes(include=['object']).nunique()[df.nunique() > 2].index.tolist()

# Binary columns (object type columns with exactly two unique values)
binary_columns = df.select_dtypes(include=['object']).nunique()[df.nunique() == 2].index.tolist()

# Print out the lists of columns
print("Numerical columns:", num_columns)
print("Categorical columns:", categorical_columns)
print("Binary columns:", binary_columns)


Numerical columns: ['months_as_customer', 'age', 'policy_deductable', 'policy_annual_premium', 'umbrella_limit', 'incident_hour_of_the_day', 'number_of_vehicles_involved', 'bodily_injuries', 'total_claim_amount']
Categorical columns: ['policy_state', 'policy_csl', 'insured_education_level', 'insured_occupation', 'insured_relationship', 'incident_type', 'collision_type', 'incident_severity', 'incident_state', 'incident_city']
Binary columns: ['insured_sex', 'property_damage', 'police_report_available', 'fraud_reported']


In [11]:
df[binary_columns].isna().sum()

insured_sex                  0
property_damage            360
police_report_available    343
fraud_reported               0
dtype: int64

In [12]:
df[num_columns].isna().sum()

months_as_customer             0
age                            0
policy_deductable              0
policy_annual_premium          0
umbrella_limit                 0
incident_hour_of_the_day       0
number_of_vehicles_involved    0
bodily_injuries                0
total_claim_amount             0
dtype: int64

In [13]:
df[categorical_columns ].isna().sum()

policy_state                 0
policy_csl                   0
insured_education_level      0
insured_occupation           0
insured_relationship         0
incident_type                0
collision_type             178
incident_severity            0
incident_state               0
incident_city                0
dtype: int64

In [14]:
from sklearn.impute import SimpleImputer

# Create imputers for categorical and binary columns
imputer = SimpleImputer(strategy='most_frequent')

# Apply imputer to categorical columns with missing values
df[categorical_columns] = imputer.fit_transform(df[categorical_columns])

# Apply imputer to binary columns with missing values
df[binary_columns] = imputer.fit_transform(df[binary_columns])

# Check for any remaining missing values
print(df[categorical_columns].isna().sum())
print(df[binary_columns].isna().sum())


policy_state               0
policy_csl                 0
insured_education_level    0
insured_occupation         0
insured_relationship       0
incident_type              0
collision_type             0
incident_severity          0
incident_state             0
incident_city              0
dtype: int64
insured_sex                0
property_damage            0
police_report_available    0
fraud_reported             0
dtype: int64


In [15]:
df[categorical_columns ].isna().sum()

policy_state               0
policy_csl                 0
insured_education_level    0
insured_occupation         0
insured_relationship       0
incident_type              0
collision_type             0
incident_severity          0
incident_state             0
incident_city              0
dtype: int64

In [16]:
import pandas as pd


# Custom mapping for binary columns
binary_mapping = {
    'insured_sex': {'FEMALE': 0, 'MALE': 1},
    'property_damage': {'NO': 0, 'YES': 1},
    'police_report_available': {'NO': 0, 'YES': 1},
    'fraud_reported': {'N': 0, 'Y': 1}
}

# Apply binary mapping
for column, mapping in binary_mapping.items():
    if column in binary_columns:
        df[column] = df[column].map(mapping)

# Custom mapping for ordinal categorical columns (if applicable)
ordinal_mapping = {
    'policy_csl': {'100/300': 1, '250/500': 2.5, '500/1000': 5},
    'insured_education_level': {'High School': 1, 'College': 2, 'Associate': 3, 'Masters': 4, 'JD': 5, 'MD': 6, 'PhD': 7},
    'incident_severity': {'Trivial Damage': 1, 'Minor Damage': 2, 'Major Damage': 3, 'Total Loss': 4}
}

# Apply ordinal mapping
for column, mapping in ordinal_mapping.items():
    if column in categorical_columns:
        df[column] = df[column].map(mapping)

# Apply one-hot encoding for the remaining categorical columns
from sklearn.preprocessing import OneHotEncoder
columns_to_one_hot = [col for col in categorical_columns if col not in ordinal_mapping]

df = pd.get_dummies(df, columns=columns_to_one_hot, drop_first=True)


In [17]:
# Display the first few rows of the transformed dataframe
df.head()

,months_as_customer,age,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_sex,insured_education_level,incident_severity,incident_hour_of_the_day,...,incident_state_PA,incident_state_SC,incident_state_VA,incident_state_WV,incident_city_Columbus,incident_city_Hillsdale,incident_city_Northbend,incident_city_Northbrook,incident_city_Riverwood,incident_city_Springfield
0,328,48,2.5,1000,1406.91,0,1,6,3,5,...,False,True,False,False,True,False,False,False,False,False
1,228,42,2.5,2000,1197.22,5000000,1,6,2,8,...,False,False,True,False,False,False,False,False,True,False
2,134,29,1.0,2000,1413.14,5000000,0,7,2,7,...,False,False,False,False,True,False,False,False,False,False
3,256,41,2.5,2000,1415.74,6000000,0,7,3,5,...,False,False,False,False,False,False,False,False,False,False
4,228,44,5.0,1000,1583.91,6000000,1,3,2,20,...,False,False,False,False,False,False,False,False,False,False


In [18]:
df['fraud_reported']

0      1
1      1
2      0
3      1
4      0
      ..
995    0
996    0
997    0
998    0
999    0
Name: fraud_reported, Length: 1000, dtype: int64

In [19]:
# Separate the features and the target variable (fraud_reported)
X = df.drop('fraud_reported', axis=1)  # Features
y = df['fraud_reported']  # Target


In [20]:
X.isna().sum()

months_as_customer                        0
age                                       0
policy_csl                                0
policy_deductable                         0
policy_annual_premium                     0
umbrella_limit                            0
insured_sex                               0
insured_education_level                   0
incident_severity                         0
incident_hour_of_the_day                  0
number_of_vehicles_involved               0
property_damage                           0
bodily_injuries                           0
police_report_available                   0
total_claim_amount                        0
policy_state_IN                           0
policy_state_OH                           0
insured_occupation_armed-forces           0
insured_occupation_craft-repair           0
insured_occupation_exec-managerial        0
insured_occupation_farming-fishing        0
insured_occupation_handlers-cleaners      0
insured_occupation_machine-op-in

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score


In [22]:
# Step 1: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [23]:
scaler = StandardScaler()

In [24]:
# Scale the numerical features in training and testing datasets
X_train[num_columns] = scaler.fit_transform(X_train[num_columns])
X_test[num_columns] = scaler.transform(X_test[num_columns])

In [25]:
# Step 3: Build the Random Forest Classifier
model = RandomForestClassifier(random_state=42, n_estimators=100)

In [26]:
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [27]:
# Evaluate the model
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.725

Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.97      0.84       145
           1       0.50      0.07      0.13        55

    accuracy                           0.73       200
   macro avg       0.62      0.52      0.48       200
weighted avg       0.67      0.72      0.64       200

